## Facility Location

with integer programming and Gurobi

Source: http://examples.gurobi.com/facility-location/

通用 facility location problem 生产问题，包括供应链，物流，运输问题等。 交互式例子为英国通过库房供应超级市场的选址问题。

In this example we'll solve a simple facility location problem: where to build warehouses to supply a large number of supermarkets.

We'll construct a mathematical model of the business problem, implement this model in Gurobi's Python interface, and compute and visualize an optimal solution.

Although your own business may not involve supermarkets, the same basic techniques used in this example can be used for many other applications in supply chain, logistics and transportation.

<img height=400 width=600 src=http://examples.gurobi.com/facility-location/screenshot.png>


## Problem Description


超市位置已经决定， 但是仓储位置还待选择。

A large supermarket chain in the UK needs to build warehouses for a set of supermarkets it is opening in Northern England. The locations of the supermarkets have been decided, but the locations of the warehouses are yet to be chosen.

有些候选位置待考虑。

Several good candidate locations for the warehouses have been determined, but it remains to decide how many warehouses to open and at which candidate locations to build them.

A typical warehouse. Source: Axisadman - CC BY-SA 3.0
Opening many warehouses would be advantageous as this would reduce the average distance a truck has to drive from warehouse to supermarket, and hence reduce the delivery cost. However, opening a warehouse is costly.

We will use Gurobi to find the optimal tradeoff between delivery cost and the cost of building new facilities.


## Implementation

In [1]:
from gurobipy import *
import math

def distance(a,b):
  dx = a[0] - b[0]
  dy = a[1] - b[1]
  return math.sqrt(dx*dx + dy*dy)

# Problem data
clients = [[0, 1.5],[2.5, 1.2]]
facilities = [[0,0],[0,1],[0,1],
              [1,0],[1,1],[1,2],
              [2,0],[2,1],[2,2]]
charge = [3,2,3,1,3,3,4,3,2]

numFacilities = len(facilities)
numClients = len(clients)

m = Model()

# Add variables
x = {}
y = {}
d = {} # Distance matrix (not a variable)
alpha = 1

for j in range(numFacilities):
  x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)

for i in range(numClients):
  for j in range(numFacilities):
    y[(i,j)] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="t%d,%d" % (i,j))
    d[(i,j)] = distance(clients[i], facilities[j])

m.update()

# Add constraints
for i in range(numClients):
  for j in range(numFacilities):
    m.addConstr(y[(i,j)] <= x[j])

for i in range(numClients):
  m.addConstr(quicksum(y[(i,j)] for j in range(numFacilities)) == 1)

m.setObjective( quicksum(charge[j]*x[j] + quicksum(alpha*d[(i,j)]*y[(i,j)]
                for i in range(numClients)) for j in range(numFacilities)) )

m.optimize()

Optimize a model with 20 rows, 27 columns and 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)

Root relaxation: objective 4.723713e+00, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.7237129    4.72371  0.00%     -    0s

Explored 0 nodes (15 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 4.72371 
Pool objective bound 4.72371

Optimal solution found (tolerance 1.00e-04)
Best objective 4.723712908962e+00, best bound 4.723712908962e+00, gap 0.0000%


## Optimal Result


In [2]:
for v in m.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

x3 1.000000
t0,3 1.000000
t1,3 1.000000


## Live Demo

UK英国 GeoLytix 公司的供应链数据。

Below is a visualization of our example. We are using the location data from GeoLytix for a large supermarket chain in the UK, and visualizing its outlets in Northern England.

有些仓库地址已经在候选当中。

The supermarkets are represented by: 

By clicking the map you can add potential warehouse locations. These are represented by: 

点击以可追加新的仓库地址。

Click "Compute Warehouse Locations" to find the locations where warehouses will be built using Gurobi. These will be represented by: 

A few potential warehouse locations have already been set up, but you can add more by clicking the screen.

运输费用为 3 pounds/mile。

The cost of transportation is 3 pounds/mile. You can use the slider to vary the cost of building a warehouse. When the cost is low, many facilities will be built. When the cost is high, it will dominate the transportation cost and there will be fewer facilities with greater driving distances.


构建仓库的费用从 0.5-3 million pounds；所以直觉上来说就是仓库费用越高，建的数量越少。

Cost to build warehouse:
